# TDSP Stage 3 – Modeling

## Proyek: Segmentasi Pelanggan Online Retail Berbasis RFM (Spark)

Stage ini berfokus pada **feature engineering RFM**, **transformasi data**, dan **clustering pelanggan** menggunakan Apache Spark.
Hasil dari stage ini adalah segmen pelanggan yang bermakna dan dapat diinterpretasikan secara bisnis.

## 3.1 Tujuan Modeling

Tujuan utama tahap modeling adalah:

1. Mengubah data transaksi menjadi **fitur RFM** pada level pelanggan.
2. Menyiapkan dataset yang siap untuk algoritma clustering.
3. Mengelompokkan pelanggan berdasarkan kemiripan perilaku transaksi.
4. Menghasilkan segmentasi yang **interpretable** dan **actionable** bagi bisnis.

Pendekatan yang digunakan bersifat **unsupervised learning**, karena tidak terdapat label target.

In [124]:
# ==============================================================
# Windows Hadoop Fix (WAJIB untuk write Parquet)
# ==============================================================

import os

os.environ["HADOOP_HOME"] = r"C:\hadoop"
os.environ["hadoop.home.dir"] = r"C:\hadoop"
os.environ["PATH"] += r";C:\hadoop\bin"

In [125]:
# ==============================================================
# 3.1 Spark Session Initialization
# ==============================================================

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("RFM-Customer-Segmentation")
    .getOrCreate()
)


## 3.2 Persiapan Data Modeling

Dataset yang digunakan adalah hasil filter dari TDSP Stage 2.
Langkah awal:
- Pastikan kolom tanggal dalam format timestamp
- Hitung nilai transaksi (Amount)
- Tentukan *reference date* untuk perhitungan Recency

In [126]:
# ==============================================================
# 3.2 Load Data & Basic Preparation (FINAL FIX)
# ==============================================================

from pyspark.sql.functions import col, to_timestamp

DATA_PATH = "../datasets/online_retail_stage2.csv.gz"

# --------------------------------------------------------------
# Load data
# --------------------------------------------------------------
df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(DATA_PATH)
)

# --------------------------------------------------------------
# FIX: Parse InvoiceDate dengan format eksplisit
# Dataset Online Retail umumnya: "12/1/2010 8:26"
# --------------------------------------------------------------
df = (
    df
    .withColumn(
        "InvoiceDate",
        to_timestamp(col("InvoiceDate"), "M/d/yyyy H:mm")
    )
    .withColumn("CustomerID", col("CustomerID").cast("string"))
    .withColumn("Quantity", col("Quantity").cast("integer"))
    .withColumn("UnitPrice", col("UnitPrice").cast("double"))
)

# --------------------------------------------------------------
# Ambil transaksi VALID untuk RFM
# (business rule: pembelian aktual)
# --------------------------------------------------------------
df_valid = df.filter(
    (col("Quantity") > 0) &
    (col("UnitPrice") > 0) &
    col("CustomerID").isNotNull() &
    col("InvoiceDate").isNotNull()   # ⬅️ KRUSIAL
)

df_valid.cache()

print("Valid transactions:", df_valid.count())

# --------------------------------------------------------------
# Sanity check (WAJIB sekali)
# --------------------------------------------------------------
df_valid.select("InvoiceDate").show(5)

Valid transactions: 354321
+-------------------+
|        InvoiceDate|
+-------------------+
|2010-12-01 08:26:00|
|2010-12-01 08:26:00|
|2010-12-01 08:26:00|
|2010-12-01 08:26:00|
|2010-12-01 08:26:00|
+-------------------+
only showing top 5 rows



## 3.3 Feature Engineering – Monetary

**Monetary** merepresentasikan total nilai uang yang dibelanjakan pelanggan.

Langkah:
- Hitung `Amount = Quantity × Price`
- Agregasi total Amount per pelanggan (PostCode)

In [127]:
# ==============================================================
# 3.3 Feature Engineering – Monetary
# ==============================================================

from pyspark.sql.functions import sum as spark_sum

df_valid = df_valid.withColumn(
    "Amount", col("Quantity") * col("UnitPrice")
)

monetary_df = (
    df_valid
    .groupBy("CustomerID")
    .agg(spark_sum("Amount").alias("Monetary"))
)

monetary_df.show(5)

+----------+------------------+
|CustomerID|          Monetary|
+----------+------------------+
|   15039.0|19914.439999999966|
|   13178.0| 5725.470000000003|
|   16553.0| 5719.819999999999|
|   17786.0|            278.74|
|   12891.0|             331.0|
+----------+------------------+
only showing top 5 rows



## 3.4 Feature Engineering – Recency & Frequency

- **Recency**: jarak waktu (hari) antara transaksi terakhir pelanggan dengan tanggal referensi
- **Frequency**: jumlah transaksi unik (Invoice) per pelanggan


In [128]:
# ==============================================================
# 3.4 Feature Engineering – Recency & Frequency
# ==============================================================

from pyspark.sql.functions import (
    max as spark_max,
    countDistinct,
    datediff,
    lit,
    to_date,
    col
)

# Reference date = transaksi terakhir di dataset valid
ref_date = df_valid.agg(spark_max("InvoiceDate")).collect()[0][0]

rf_df = (
    df_valid
    # turunkan ke level tanggal agar tidak over-count
    .withColumn("InvoiceDay", to_date(col("InvoiceDate")))
    .groupBy("CustomerID")
    .agg(
        spark_max("InvoiceDate").alias("LastPurchaseDate"),
        countDistinct("InvoiceDay").alias("Frequency")
    )
    .withColumn(
        "Recency",
        datediff(lit(ref_date), col("LastPurchaseDate"))
    )
    .drop("LastPurchaseDate")
)

# Gabungkan RFM
from pyspark.sql.functions import coalesce, lit

rfm_df = (
    rf_df
    .join(monetary_df, on="CustomerID", how="left")
    .withColumn("Monetary", coalesce(col("Monetary"), lit(0.0)))
)

print("Total customers with RFM:", rfm_df.count())
rfm_df.show(5)

Total customers with RFM: 3920
+----------+---------+-------+------------------+
|CustomerID|Frequency|Recency|          Monetary|
+----------+---------+-------+------------------+
|   15039.0|       42|      9|19914.439999999966|
|   17966.0|        4|     37|           1098.43|
|   16553.0|        6|    163| 5719.819999999999|
|   17955.0|        3|    198|             557.3|
|   13178.0|       11|     26| 5725.470000000003|
+----------+---------+-------+------------------+
only showing top 5 rows



## 3.5 Pemeriksaan Distribusi RFM

Distribusi RFM biasanya **skewed**, khususnya Monetary.
Pemeriksaan statistik deskriptif penting sebelum scaling dan clustering.

In [129]:
# ==============================================================
# 3.5 Pemeriksaan Distribusi RFM
# ==============================================================

rfm_df.select(
    "Recency",
    "Frequency",
    "Monetary"
).summary().show()

+-------+-----------------+-----------------+------------------+
|summary|          Recency|        Frequency|          Monetary|
+-------+-----------------+-----------------+------------------+
|  count|             3920|             3920|              3920|
|   mean|91.74209183673469|             3.85|1864.3856005102004|
| stddev|99.53348473774295|5.713358015316058| 7482.817476870354|
|    min|                0|                1|              3.75|
|    25%|               17|                1|300.03999999999996|
|    50%|               50|                2| 651.8200000000002|
|    75%|              142|                4|1575.8900000000003|
|    max|              373|              113|259657.30000000005|
+-------+-----------------+-----------------+------------------+



## 3.6 Feature Scaling

Karena perbedaan skala antar fitur RFM cukup besar,
maka diperlukan **normalisasi / scaling** sebelum clustering.

Metode yang digunakan:
- `StandardScaler` (mean = 0, std = 1)

In [130]:
# ==============================================================
# 3.6 Feature Scaling (Option 1: Proper Cleaning)
# ==============================================================

from pyspark.sql.functions import col

rfm_clean = (
    rfm_df
    .filter(col("Recency").isNotNull())
    .filter(col("Frequency") >= 1)
    .filter(col("Monetary") > 0)
)

print("RFM rows after cleaning:", rfm_clean.count())
rfm_clean.show(5)

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["Recency", "Frequency", "Monetary"],
    outputCol="features_vec"
)

from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline

scaler = StandardScaler(
    inputCol="features_vec",
    outputCol="scaled_features",
    withMean=True,
    withStd=True
)

pipeline_prep = Pipeline(stages=[assembler, scaler])

prep_model = pipeline_prep.fit(rfm_clean)
rfm_scaled = prep_model.transform(rfm_clean)

rfm_scaled.select(
    "CustomerID",
    "Recency",
    "Frequency",
    "Monetary",
    "scaled_features"
).show(5, truncate=False)

RFM rows after cleaning: 3920
+----------+---------+-------+------------------+
|CustomerID|Frequency|Recency|          Monetary|
+----------+---------+-------+------------------+
|   15039.0|       42|      9|19914.439999999966|
|   17966.0|        4|     37|           1098.43|
|   16553.0|        6|    163| 5719.819999999999|
|   17955.0|        3|    198|             557.3|
|   13178.0|       11|     26| 5725.470000000003|
+----------+---------+-------+------------------+
only showing top 5 rows

+----------+-------+---------+------------------+--------------------------------------------------------------+
|CustomerID|Recency|Frequency|Monetary          |scaled_features                                               |
+----------+-------+---------+------------------+--------------------------------------------------------------+
|15039.0   |9      |42       |19914.439999999966|[-0.8312990553354861,6.677334047285251,2.412200278208456]     |
|17966.0   |37     |4        |1098.43      

## 3.7 Clustering Pelanggan (K-Means)

Algoritma **K-Means** digunakan untuk mengelompokkan pelanggan berdasarkan fitur RFM.

Jumlah cluster ditentukan berdasarkan:
- Interpretabilitas bisnis
- Eksperimen awal

Pada tahap ini digunakan **k = 5**, sesuai praktik umum segmentasi RFM.

In [131]:
# ==============================================================
# 3.7 Clustering Pelanggan (K-Means)
# ==============================================================

from pyspark.ml.clustering import KMeans

# Jumlah cluster (praktik umum RFM)
K = 5

kmeans = KMeans(
    featuresCol="scaled_features",
    predictionCol="Cluster",
    k=K,
    seed=42
)

# Train model
kmeans_model = kmeans.fit(rfm_scaled)

# Assign cluster ke setiap customer
rfm_clustered = kmeans_model.transform(rfm_scaled)

rfm_clustered.select(
    "CustomerID",
    "Recency",
    "Frequency",
    "Monetary",
    "Cluster"
).show(10)

+----------+-------+---------+------------------+-------+
|CustomerID|Recency|Frequency|          Monetary|Cluster|
+----------+-------+---------+------------------+-------+
|   15039.0|      9|       42|19914.439999999966|      3|
|   17966.0|     37|        4|           1098.43|      0|
|   16553.0|    163|        6| 5719.819999999999|      2|
|   17955.0|    198|        3|             557.3|      2|
|   13178.0|     26|       11| 5725.470000000003|      4|
|   12985.0|      0|        2|           1239.38|      0|
|   16557.0|     24|        2|281.84999999999997|      0|
|   17786.0|     85|        2|            278.74|      0|
|   13259.0|     61|        1| 292.3199999999999|      0|
|   14349.0|     10|        1|133.50000000000006|      0|
+----------+-------+---------+------------------+-------+
only showing top 10 rows



In [132]:
from pyspark.sql.functions import count

rfm_clustered.groupBy("Cluster") \
    .agg(count("CustomerID").alias("Num_Customers")) \
    .orderBy("Cluster") \
    .show()

+-------+-------------+
|Cluster|Num_Customers|
+-------+-------------+
|      0|         2603|
|      1|            3|
|      2|          962|
|      3|           25|
|      4|          327|
+-------+-------------+



In [133]:
from pyspark.sql.functions import avg, round as spark_round

cluster_profile = (
    rfm_clustered
    .groupBy("Cluster")
    .agg(
        spark_round(avg("Recency"), 1).alias("Avg_Recency"),
        spark_round(avg("Frequency"), 1).alias("Avg_Frequency"),
        spark_round(avg("Monetary"), 2).alias("Avg_Monetary"),
        count("CustomerID").alias("Num_Customers")
    )
    .orderBy("Cluster")
)

cluster_profile.show()

+-------+-----------+-------------+------------+-------------+
|Cluster|Avg_Recency|Avg_Frequency|Avg_Monetary|Num_Customers|
+-------+-----------+-------------+------------+-------------+
|      0|       44.9|          3.0|     1114.35|         2603|
|      1|        2.7|         18.3|    207560.2|            3|
|      2|      247.0|          1.5|       493.0|          962|
|      3|       17.8|         45.2|     45946.6|           25|
|      4|       14.3|         14.1|     6611.98|          327|
+-------+-----------+-------------+------------+-------------+



## 3.8 Interpretasi Awal Cluster

Langkah awal interpretasi:
- Hitung statistik RFM per cluster
- Bandingkan karakteristik antar cluster

Interpretasi ini akan menjadi dasar rekomendasi bisnis pada stage selanjutnya.

In [134]:
# ==============================================================
# 3.8 Cluster Profiling (RFM Statistics)
# ==============================================================

from pyspark.sql.functions import avg, count, round as spark_round

cluster_profile = (
    rfm_clustered
    .groupBy("Cluster")
    .agg(
        count("CustomerID").alias("Num_Customers"),
        spark_round(avg("Recency"), 1).alias("Avg_Recency"),
        spark_round(avg("Frequency"), 1).alias("Avg_Frequency"),
        spark_round(avg("Monetary"), 2).alias("Avg_Monetary")
    )
    .orderBy("Cluster")
)

cluster_profile.show()

+-------+-------------+-----------+-------------+------------+
|Cluster|Num_Customers|Avg_Recency|Avg_Frequency|Avg_Monetary|
+-------+-------------+-----------+-------------+------------+
|      0|         2603|       44.9|          3.0|     1114.35|
|      1|            3|        2.7|         18.3|    207560.2|
|      2|          962|      247.0|          1.5|       493.0|
|      3|           25|       17.8|         45.2|     45946.6|
|      4|          327|       14.3|         14.1|     6611.98|
+-------+-------------+-----------+-------------+------------+



In [ ]:
pass

## Ringkasan Stage 3

Pada TDSP Stage 3 ini, kita telah:
- Melakukan feature engineering RFM menggunakan Spark
- Melakukan scaling fitur untuk clustering
- Mengelompokkan pelanggan menggunakan K-Means
- Menghasilkan segmentasi pelanggan awal

Tahap berikutnya adalah **TDSP Stage 4 – Deployment**,
yang berfokus pada penyajian hasil, insight bisnis, dan output yang siap dikonsumsi stakeholder.